In [ ]:
# ============================================================
# IMPORTS
# ============================================================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

# ============================================================
# CONFIG
# ============================================================
SEQ_LEN = 30
BATCH_SIZE = 64
EPOCHS = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_RUL = 125

# ============================================================
# LOAD DATA (FD001)
# ============================================================
cols = (
    ["unit", "cycle"] +
    [f"op_{i}" for i in range(1, 4)] +
    [f"s_{i}" for i in range(1, 22)]
)

train_df = pd.read_csv("train_FD001.txt", delim_whitespace=True, header=None, names=cols)
test_df  = pd.read_csv("test_FD001.txt", delim_whitespace=True, header=None, names=cols)
rul_df   = pd.read_csv("RUL_FD001.txt", header=None, names=["RUL"])

# ============================================================
# RUL CALCULATION
# ============================================================
max_cycles = train_df.groupby("unit")["cycle"].max()
train_df["RUL"] = train_df.apply(
    lambda r: max_cycles[r["unit"]] - r["cycle"], axis=1
)
train_df["RUL"] = train_df["RUL"].clip(upper=MAX_RUL)

# ============================================================
# FEATURE SCALING
# ============================================================
features = cols[2:]
scaler = MinMaxScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features]  = scaler.transform(test_df[features])

# ============================================================
# TRAIN / VALIDATION SPLIT (ENGINE-WISE)
# ============================================================
engines = train_df["unit"].unique()
train_eng, val_eng = train_test_split(engines, test_size=0.2, random_state=42)

train_df = train_df[train_df["unit"].isin(train_eng)]
val_df   = train_df[train_df["unit"].isin(val_eng)]

# ============================================================
# SEQUENCE GENERATION
# ============================================================
def generate_sequences(df, seq_len, is_test=False):
    X, y = [], []
    for unit in df["unit"].unique():
        u_df = df[df["unit"] == unit]
        data = u_df[features].values

        if is_test:
            X.append(data[-seq_len:])
        else:
            rul = u_df["RUL"].values
            for i in range(len(data) - seq_len):
                X.append(data[i:i + seq_len])
                y.append(rul[i + seq_len - 1])

    return np.array(X), np.array(y)

X_train, y_train = generate_sequences(train_df, SEQ_LEN)
X_val, y_val     = generate_sequences(val_split_df, SEQ_LEN)
X_test, _        = generate_sequences(test_df, SEQ_LEN, is_test=True)

# ============================================================
# DATASET
# ============================================================
class CMAPSSDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx]) if self.y is not None else self.X[idx]

train_loader = DataLoader(CMAPSSDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(CMAPSSDataset(X_val, y_val), batch_size=BATCH_SIZE)
test_loader  = DataLoader(CMAPSSDataset(X_test), batch_size=BATCH_SIZE)

# ============================================================
# LSTM MODEL
# ============================================================
class RULLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1]).squeeze(1)

# ============================================================
# FITNESS FUNCTION FOR GWO
# ============================================================
def fitness_function(params):
    hidden_dim, num_layers, dropout, lr = params
    hidden_dim = int(hidden_dim)
    num_layers = max(1, int(num_layers))  # Ensure num_layers >= 1

    model = RULLSTM(len(features), hidden_dim, num_layers, dropout).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    # Train few epochs for speed
    for _ in range(5):
        model.train()
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            opt.zero_grad()
            loss = loss_fn(model(X), y)
            loss.backward()
            opt.step()

    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            preds.append(model(X).cpu().numpy())
            true.append(y.cpu().numpy())

    return np.sqrt(mean_squared_error(np.concatenate(true), np.concatenate(preds)))

# ============================================================
# GREY WOLF OPTIMIZER
# ============================================================
def grey_wolf_optimizer(fitness_fn, bounds, wolves=6, iters=8):
    dim = len(bounds)
    pop = np.random.rand(wolves, dim)

    for i in range(dim):
        low, high = bounds[i]
        pop[:, i] = low + pop[:, i] * (high - low)

    alpha = beta = delta = None
    a_score = b_score = d_score = np.inf

    for t in range(iters):
        for wolf in pop:
            score = fitness_fn(wolf)
            if score < a_score:
                alpha, a_score = wolf.copy(), score
            elif score < b_score:
                beta, b_score = wolf.copy(), score
            elif score < d_score:
                delta, d_score = wolf.copy(), score

        a = 2 - t * (2 / iters)

        for i in range(wolves):
            for j in range(dim):
                X1 = alpha[j] - a * np.random.rand() * abs(alpha[j] - pop[i][j])
                X2 = beta[j]  - a * np.random.rand() * abs(beta[j] - pop[i][j])
                X3 = delta[j] - a * np.random.rand() * abs(delta[j] - pop[i][j])
                pop[i][j] = (X1 + X2 + X3) / 3

    return alpha, a_score

# ============================================================
# RUN GWO
# ============================================================
bounds = [
    (32, 128),   # hidden_dim
    (1, 3),      # num_layers
    (0.1, 0.5),  # dropout
    (1e-4, 1e-3) # learning rate
]

best_params, best_rmse = grey_wolf_optimizer(fitness_function, bounds)

print("\nBest Hyperparameters:", best_params)
print("Best Val RMSE:", best_rmse)

# ============================================================
# FINAL TRAINING
# ============================================================
hidden, layers, dropout, lr = best_params
model = RULLSTM(len(features), int(hidden), int(layers), dropout).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

for epoch in range(EPOCHS):
    model.train()
    losses = []
    for X, y in train_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(X), y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    print(f"Epoch {epoch+1} | Train MSE: {np.mean(losses):.4f}")

# ============================================================
# TEST EVALUATION
# ============================================================
model.eval()
preds = []
with torch.no_grad():
    for X in test_loader:
        X = X.to(DEVICE)
        preds.append(model(X).cpu().numpy())

test_preds = np.concatenate(preds)
test_rmse = np.sqrt(mean_squared_error(rul_df["RUL"].values, test_preds))

print("\nTEST RMSE:", test_rmse)

# ============================================================
# HEALTH DEGRADATION PLOT
# ============================================================
def plot_engine_health(engine_id):
    df = train_df[train_df["unit"] == engine_id]
    health = []

    with torch.no_grad():
        for i in range(len(df) - SEQ_LEN):
            seq = torch.tensor(
                df.iloc[i:i+SEQ_LEN][features].values,
                dtype=torch.float32
            ).unsqueeze(0).to(DEVICE)

            pred = model(seq).cpu().item()
            health.append(np.clip((pred / MAX_RUL) * 100, 0, 100))

    plt.plot(health)
    plt.xlabel("Cycle")
    plt.ylabel("Health (%)")
    plt.title(f"Engine {engine_id} Health Degradation")
    plt.grid()
    plt.show()

plot_engine_health(1)


/var/folders/kw/2mknhk896xn3vj_txfhlh5v40000gn/T/ipykernel_44149/3294785398.py:33: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_df = pd.read_csv("train_FD001.txt", delim_whitespace=True, header=None, names=cols)
/var/folders/kw/2mknhk896xn3vj_txfhlh5v40000gn/T/ipykernel_44149/3294785398.py:34: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  test_df  = pd.read_csv("test_FD001.txt", delim_whitespace=True, header=None, names=cols)
/Users/rishavkundu/Downloads/anaconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.20696624270382805 and num_layers=1
  warnings.warn(
/Users/rishavkundu/Downloads/anaconda3/lib/python3.12/site-packages/

ValueError: num_layers must be greater than zero